In [15]:
import pymongo

In [16]:
conn = pymongo.MongoClient() #몽고db 서버에 연결

In [17]:
answers=conn.baenangE.answers

In [18]:
answers.count_documents({})

11823

In [19]:
doc=answers.find_one({"daily_index":2})
doc

{'_id': ObjectId('62abe652f9693add28944455'),
 'daily_index': 2,
 'A': '여행은 언제나 좋죠.'}

In [20]:
answers.find_one({"daily_index":100}).get('A')

'밥 사줄 친구를 찾아 보세요~'

In [21]:
import joblib

In [22]:
# tfidf파일 load
tfidf_daily_life=joblib.load('tfidf_chatbot_daily_life.pkl') # 입력받은 문장을 벡터화 하기 위한 파일
daily_life=joblib.load('vetorized_chatbot_daily_life.pkl') # 기존 질문 문장들을 벡터화한 파일, 위에서 입력받은 문장과 유사도를 비교

In [23]:
from konlpy.tag import Okt
t = Okt()

In [24]:
# new_post 와 contents의 거리 구하는 함수 생성
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [25]:
def baenangE_daily_life():
    str=input("입력 >>")
    
    # test text 벡터화
    # 형태소 분석 후 띄어쓰기로 구분하여 하나의 문장으로 만들기
    new_post_tokens = [t.morphs(row) for row in [str]]

    new_post_for_vectorize = []

    for content in new_post_tokens:
        sentence = ''
        for word in content:
            sentence = sentence + ' ' + word

        new_post_for_vectorize.append(sentence)

    new_post_for_vectorize
    new_post_vec = tfidf_daily_life.transform(new_post_for_vectorize)
  
    # 다른 결과를 얻을 수 있음
    best_doc = None
    best_dist = 65535
    best_i = None

    for i in range(0, 11823):
        post_vec = daily_life.getrow(i)

        # 함수호출
        d = dist_raw(post_vec, new_post_vec)

        if d<best_dist:
            best_dist = d
            best_i = i
    print("배낭이 >> ", answers.find_one({"daily_index":best_i}).get('A'))

In [26]:
baenangE_daily_life()

입력 >>밥 뭐먹지
배낭이 >>  색다른걸 드셔보세요.


In [27]:
baenangE_daily_life()

입력 >>위로좀해줘
배낭이 >>  제게 기대세요.


In [28]:
baenangE_daily_life()

입력 >>짜증난다
배낭이 >>  짜증날 땐 짜장면
